In [10]:
import json
from IPython.display import display, HTML
from datetime import datetime, timedelta

# Load the data from the file
with open('rounded_structured_metal_data.json', 'r') as file:
    data = json.load(file)
    metals_data = data["Metals"]

# Define periods for price changes
periods = {
    "1 Day": 1,
    "1 Week": 7,
    "1 Month": 30,
    "3 Months": 90,
    "1 Year": 365
}

# Process data to get percentage and numerical changes
dashboard_data = []
for metal in metals_data:
    metal_name = metal["Metal_Name"]
    unit = metal["Unit"]
    prices = metal["Prices"]
    
    # Sort prices by date
    prices = sorted(prices, key=lambda x: datetime.strptime(x["Date_Collected"], '%Y-%m-%d'), reverse=True)
    
    current_price = prices[0]["Price"]
    price_changes = {}
    current_date = datetime.strptime(prices[0]["Date_Collected"], '%Y-%m-%d')
    
    for label, days in periods.items():
        target_date = current_date - timedelta(days=days)
        
        # Find price closest to target date
        price_on_target_date = next((p["Price"] for p in prices if datetime.strptime(p["Date_Collected"], '%Y-%m-%d') <= target_date), None)
        
        if price_on_target_date:
            change_value = current_price - price_on_target_date
            percent_change = (change_value / price_on_target_date) * 100
            price_changes[label] = {
                "Numerical": round(change_value, 2),
                "Percentage": f"{round(percent_change, 2)}%"
            }
        else:
            price_changes[label] = {
                "Numerical": "-",
                "Percentage": "-"
            }
    
    dashboard_data.append({
        "Metal Name": metal_name,
        "Unit": unit,
        "Current Price": current_price,
        "Price Changes": price_changes
    })

# Convert dashboard data to an HTML table with conditional formatting
def create_dashboard_table(data):
    html = ["<table width=100%>"]
    html.append("<tr>")
    headers = ["Metal Name", "Unit", "Current Price"]
    for period in ["1 Day", "1 Week", "1 Month", "3 Months", "1 Year"]:
        headers.extend([f"{period} Change (Num)", f"{period} Change (%)"])
    html.append("".join([f"<th>{header}</th>" for header in headers]))
    html.append("</tr>")
    
    for metal_data in data:
        html.append("<tr>")
        html.append(f"<td>{metal_data['Metal Name']}</td>")
        html.append(f"<td>{metal_data['Unit']}</td>")
        html.append(f"<td>${metal_data['Current Price']:,.2f}</td>")  # Format as dollar
        for period_change in metal_data["Price Changes"].values():
            # Determine color based on value
            color = ""
            if period_change["Numerical"] != "-" and period_change["Numerical"] != "N/A":
                if float(period_change["Numerical"]) < 0:
                    color = "red"
                else:
                    color = "green"
            html.append(f"<td style='color:{color}'>${period_change['Numerical'] if period_change['Numerical'] not in ['-', 'N/A'] else period_change['Numerical']}</td>")  # Format as dollar and apply color
            html.append(f"<td style='color:{color}'>{period_change['Percentage']}</td>")  # Apply color
        html.append("</tr>")
    
    html.append("</table>")
    return ''.join(html)

with open('metal_dashboard.html', 'w') as file:
    file.write(dashboard_html)